<a href="https://colab.research.google.com/github/dubovik-alexander/Netology-RSML-12/blob/main/HW2_RecSystems_ColFil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание по теме «Рекомендации на основе содержания»

### Задание

1. Пакет SURPRISE:
2. используйте данные MovieLens 1M (https://grouplens.org/datasets/movielens/latest/).
3. можно использовать любые модели из пакета
4. получите RMSE на тестовом сете 0,87 и ниже

**Комментарий преподавателя**:\
В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.

In [1]:
import pandas as pd
import numpy as np
from tqdm import notebook

In [2]:
# !pip install surprise

In [3]:
from surprise import KNNWithMeans, KNNBasic, SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [4]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

In [5]:
!unzip MovieLens.zip

##### Загружаем данные и собираем датасет(фильм-рейтинг)

In [6]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [7]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


##### Используем средства SURPRISE для перевода pandas датафрейма в нужный формат

In [8]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [9]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [10]:
ratings.rating.min()

0.5

In [11]:
ratings.rating.max()

5.0

In [12]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [13]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

##### Отбираем алгоритм/алгоритмы из SURPRISE, которые будем обучать

Аccuracy metrics:

* rmse - Compute RMSE (Root Mean Squared Error)
* mse - Compute MSE (Mean Squared Error)
* mae - Compute MAE (Mean Absolute Error)
* fcp - Compute FCP (Fraction of Concordant Pairs)


In [14]:
# Использование алгоритма SVD (Singular Value Decomposition)
algo_SVD =  SVD(n_epochs=15, verbose=False, biased=True, n_factors=15)
# algo =  SVD(n_epochs=5, verbose=False, biased=True, n_factors=10) # слишком хорошо и не удовлетворяет условием задачи что бы RMSE было 0,87 и ниже.
# n_epochs: количество эпох обучения.
# verbose: если True, то выводит информацию о процессе обучения.
# biased: если True, то использует упрощенную версию SVD.
# n_factors: количество факторов в модели.

# Запуск 5-фолдовой кросс-валидации и вывод результатов
algo_SVD.fit(trainset)
# test_pred_svd = algo_SVD.test(testset)
# accuracy.rmse(test_pred_svd, verbose=True) # использовал для проверки

# Запуск 5-fold cross-validation и вывод
cross_validate(algo_SVD, data, measures=['RMSE', 'MSE', 'MAE', 'FCP'], cv=5, verbose=True)

Evaluating RMSE, MSE, MAE, FCP of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8746  0.8767  0.8703  0.8674  0.8740  0.8726  0.0033  
MSE (testset)     0.7648  0.7687  0.7575  0.7523  0.7639  0.7614  0.0058  
MAE (testset)     0.6731  0.6737  0.6675  0.6696  0.6723  0.6712  0.0024  
FCP (testset)     0.6755  0.6642  0.6684  0.6600  0.6673  0.6671  0.0051  
Fit time          0.80    0.80    2.76    1.34    0.69    1.28    0.78    
Test time         0.31    0.22    0.48    0.40    0.12    0.30    0.13    


{'test_rmse': array([0.87455161, 0.87672798, 0.87033236, 0.86736218, 0.87399416]),
 'test_mse': array([0.76484052, 0.76865195, 0.75747841, 0.75231715, 0.7638658 ]),
 'test_mae': array([0.67308376, 0.67374453, 0.66748963, 0.66958114, 0.67234541]),
 'test_fcp': array([0.67549424, 0.66420154, 0.66837108, 0.66000537, 0.66727844]),
 'fit_time': (0.8020503520965576,
  0.8028900623321533,
  2.7637345790863037,
  1.3429765701293945,
  0.6947519779205322),
 'test_time': (0.30801939964294434,
  0.21587252616882324,
  0.4801652431488037,
  0.39571547508239746,
  0.12037825584411621)}

In [15]:
# Использование алгоритма KNNBasic
algo_KNNB = KNNBasic(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute similarities between items
})
algo_KNNB.fit(trainset)


cross_validate(algo_KNNB, data, measures=['RMSE', 'MSE', 'MAE', 'FCP'], cv=5, verbose=True)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE, FCP of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9734  0.9622  0.9820  0.9657  0.9788  0.9724  0.0075  
MSE (testset)     0.9475  0.9257  0.9643  0.9325  0.9580  0.9456  0.0146  
MAE (testset)     0.7591  0.7493  0.7663  0.7488  0.7603  0.7568  0.0068  
FCP (testset)     0.4328  0.4334  0.4354  0.4324  0.4366  0.4341  0.0016  
Fit time          8.26    8.37    7.55    7.22    7.97    7.87    0.43    
Test time         10.22  

{'test_rmse': array([0.97339262, 0.96215878, 0.98198069, 0.96567644, 0.97879738]),
 'test_mse': array([0.94749319, 0.92574952, 0.96428608, 0.93253099, 0.95804431]),
 'test_mae': array([0.75910115, 0.74927182, 0.76634715, 0.74881372, 0.76027764]),
 'test_fcp': array([0.43284484, 0.43338017, 0.43539872, 0.43243485, 0.43664918]),
 'fit_time': (8.255648612976074,
  8.366158485412598,
  7.547303915023804,
  7.223072052001953,
  7.967855215072632),
 'test_time': (10.223539590835571,
  8.616150140762329,
  9.901300191879272,
  10.003084897994995,
  8.82258415222168)}

In [16]:
# Использование алгоритма SVDpp
# Комменатрий для себя что бы не забыть
# SVDpp (Singular Value Decomposition Plus Plus) - это алгоритм матричной факторизации, который используется в библиотеке Surprise для рекомендательных систем.
# Этот алгоритм является расширением алгоритма SVD (Singular Value Decomposition), которое учитывает неявные отзывы в данных.
# В отличие от SVD, который требует явных отзывов, SVDpp может обрабатывать неявные отзывы, что делает его полезным для рекомендательных систем,
# где отзывы пользователей могут быть неявными (например, когда пользователь просматривает товар, но не оставляет отзыв)

algo_SVDPP = SVDpp()

cross_validate(algo_SVDPP, data, measures=['RMSE', 'MSE', 'MAE', 'FCP'], cv=5, verbose=True)

Evaluating RMSE, MSE, MAE, FCP of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8624  0.8592  0.8652  0.8659  0.8579  0.8621  0.0032  
MSE (testset)     0.7437  0.7382  0.7487  0.7497  0.7360  0.7433  0.0055  
MAE (testset)     0.6616  0.6576  0.6622  0.6647  0.6547  0.6601  0.0036  
FCP (testset)     0.6707  0.6728  0.6777  0.6750  0.6762  0.6745  0.0025  
Fit time          111.20  111.31  111.29  117.80  115.16  113.35  2.69    
Test time         12.39   12.27   12.06   13.65   15.15   13.10   1.16    


{'test_rmse': array([0.86238788, 0.85918789, 0.86524909, 0.86585521, 0.85792887]),
 'test_mse': array([0.74371285, 0.73820384, 0.74865598, 0.74970525, 0.73604195]),
 'test_mae': array([0.66155835, 0.65757512, 0.66216147, 0.66470163, 0.65465205]),
 'test_fcp': array([0.67071398, 0.67284823, 0.67768485, 0.67498992, 0.67624755]),
 'fit_time': (111.20256423950195,
  111.3055899143219,
  111.29202938079834,
  117.79814267158508,
  115.15973615646362),
 'test_time': (12.39429783821106,
  12.270484447479248,
  12.058011293411255,
  13.649759292602539,
  15.15145492553711)}

In [17]:
# Использование алгоритма KNNWithMeans item_based
algo_KNMWM_ib = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute similarities between items
})
algo_KNMWM_ib.fit(trainset)

cross_validate(algo_KNMWM_ib, data, measures=['RMSE', 'MSE', 'MAE', 'FCP'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE, FCP of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9038  0.9014  0.8987  0.8993  0.9002  0.9007  0.0018  
MSE (testset)     0.8168  0.8125  0.8077  0.8087  0.8104  0.8112  0.0033  
MAE (testset)     0.6881  0.6879  0.6882  0.6884  0.6868  0.6879  0.0006  
FCP (testset)     0.6456  0.6501  0.6456  0.6541  0.6569  0.6505  0.0045  
Fit time          9.99    7.51    7.73    7.95    8.95    8.43    0.93    
Test time         10.

{'test_rmse': array([0.90379302, 0.90137715, 0.89871756, 0.89925734, 0.90022443]),
 'test_mse': array([0.81684183, 0.81248077, 0.80769326, 0.80866376, 0.81040403]),
 'test_mae': array([0.68813063, 0.68786638, 0.68823117, 0.68840181, 0.68675039]),
 'test_fcp': array([0.64559324, 0.65007594, 0.64562622, 0.65414481, 0.6568576 ]),
 'fit_time': (9.991855144500732,
  7.509260654449463,
  7.7267303466796875,
  7.9494171142578125,
  8.951449632644653),
 'test_time': (10.009212970733643,
  10.713686466217041,
  11.674769639968872,
  10.58335828781128,
  11.042923212051392)}

In [18]:
# Использование алгоритма KNNWithMeans user_based
algo_KNMWM_ub = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True  # compute similarities between users
})
algo_KNMWM_ub.fit(trainset)


cross_validate(algo_KNMWM_ub, data, measures=['RMSE', 'MSE', 'MAE', 'FCP'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE, FCP of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8932  0.9057  0.9084  0.8951  0.8994  0.9004  0.0059  
MSE (testset)     0.7978  0.8204  0.8252  0.8012  0.8090  0.8107  0.0106  
MAE (testset)     0.6841  0.6905  0.6940  0.6861  0.6877  0.6885  0.0034  
FCP (testset)     0.6578  0.6563  0.6398  0.6470  0.6407  0.6483  0.0076  
Fit time          0.18    0.30    0.21    0.24    0.22    0.23    0.04    
Test time         1.6

{'test_rmse': array([0.89320002, 0.90573749, 0.90843166, 0.89511958, 0.8994222 ]),
 'test_mse': array([0.79780627, 0.8203604 , 0.82524808, 0.80123906, 0.80896029]),
 'test_mae': array([0.68413013, 0.69050307, 0.69396018, 0.68614343, 0.68774912]),
 'test_fcp': array([0.65782833, 0.65634003, 0.63978271, 0.64697414, 0.64067605]),
 'fit_time': (0.17622876167297363,
  0.29764795303344727,
  0.20584964752197266,
  0.24220871925354004,
  0.22333335876464844),
 'test_time': (1.6113369464874268,
  2.3292336463928223,
  1.409625768661499,
  1.9201371669769287,
  1.4274804592132568)}

> *Вывод*:\
> В Задании было условие: получите RMSE на тестовом сете 0,87 и ниже \
> После тестирования разными алгоритмами, методами подбора, в том числе, самым приближенным к условию оказался алгоритмы SVD и его расширение SVDpp.